In [1]:
import gpytorch
import numpy as np

from bo_functions import best_error_advanced
from bo_functions import calc_ei_advanced
from bo_functions import LHS_Design
from bo_functions import create_y_data
from bo_functions import test_train_split
from bo_functions import ExactGPModel
from bo_functions import train_GP_model
from bo_functions import calc_GP_outputs
from bo_functions import calc_y_expected
from bo_functions import improvement_integral
from bo_functions import improvement_int_terms

from bo_plotters import plotter_adv
from bo_plotters import y_plotter_adv
from bo_plotters import stdev_plotter_adv
from bo_plotters import error_plotter_adv
from bo_plotters import ei_plotter_adv
from bo_plotters import improvement_integral_plot
from bo_plotters import improvement_int_terms_plot

In [2]:
#Set noise parameters and load csv file to generate training data
noise_std = 0.1**2
all_p = LHS_Design("LHS_Toy_3_Input.csv")
all_y = create_y_data(all_p, noise_std)

#Separate training and testing data, uses default of an 80%/20% split
train_test_data = test_train_split(all_p,all_y)

train_p = train_test_data[0] #1x19
train_y = train_test_data[1] #1x19
test_p = train_test_data[2] #1x19
test_y = train_test_data[3] #1x19
print(test_p)

tensor([[ 1.6023, -0.3062,  1.9780],
        [ 1.3769, -0.4788, -1.4939],
        [ 0.2180, -1.6312, -1.0622],
        [ 1.7776, -1.8356, -1.6141],
        [ 0.4116, -1.0924,  0.0214],
        [-0.8668,  1.2260, -1.0091]], dtype=torch.float64)


In [3]:
# initialize likelihood and model
##Assumes a homoskedastic noise model p(y | f) = f + noise
likelihood = gpytorch.likelihoods.GaussianLikelihood()

# We will use the simplest form of GP model, exact inference
#Defines our model in terms of the class parameters in bo_functions
model = ExactGPModel(train_p, train_y, likelihood)

In [4]:
#Set number of training iterations and train GP
iterations = 500
train_GP_model(model,likelihood, train_p, train_y, iterations)

In [5]:
# Get into evaluation (predictive posterior) mode
#Puts model in evaluation mode
model.eval()
#Puts likelihood in evaluation mode
likelihood.eval();

In [6]:
#Create Meshgrid

test_p1 = np.linspace(-2,2,5) #Theta 1
test_p2 = np.linspace(-2,2,5) #Theta 2
test_p3 = np.linspace(-2,2,5) #Theta 3

test_p_mesh = np.array(np.meshgrid(test_p1,test_p2,test_p3))

test_p_mesh_plot = test_p_mesh.T.reshape(-1,3)

In [9]:
#Calculates GP outputs for mean, variance, standard devaition, and y output
GP_Outputs = calc_GP_outputs(model,likelihood,test_p_mesh_plot)

model_mean = GP_Outputs[0] #1x6
model_variance= GP_Outputs[1] #1x6
model_stdev = GP_Outputs[2] #1x6
model_y = GP_Outputs[3] #1x6

#Calculates expected y
y_exp = calc_y_expected(test_p_mesh_plot, noise_std) #1x6

print("Model Mean \n", model_mean)
print("Model Variance \n", model_variance)
print("Model Standard Deviation \n", model_stdev)
print("Model y \n", model_y)
print("Y Value Expected \n", y_exp)

Model Mean 
 tensor([-1.6575, -1.2703, -0.8330, -0.6817, -0.8430, -3.4124, -3.0468, -2.1987,
        -1.4608, -1.1515, -5.8189, -5.5732, -4.2055, -2.6154, -1.5661, -7.6039,
        -7.4931, -5.7485, -3.4791, -1.8189, -7.6159, -7.5799, -5.8526, -3.5120,
        -1.7516, -1.1685, -0.5556, -0.0319, -0.0187, -0.4657, -2.5342, -1.8669,
        -0.9559, -0.4661, -0.5794, -4.5517, -3.9286, -2.5079, -1.2469, -0.7309,
        -6.1470, -5.6112, -3.7907, -1.8535, -0.7580, -6.2856, -5.8409, -4.0034,
        -1.9204, -0.6533, -0.7127,  0.1308,  0.8088,  0.7897,  0.1261, -1.3915,
        -0.3891,  0.6106,  0.8702,  0.3131, -2.5275, -1.4428, -0.0245,  0.7567,
         0.5523, -3.5064, -2.4162, -0.6545,  0.6397,  0.8315, -3.6869, -2.6964,
        -0.9133,  0.5486,  0.9409, -0.4597,  0.5654,  1.4442,  1.5330,  0.8026,
        -0.5158,  0.7628,  1.9071,  2.0994,  1.2712, -0.7652,  0.7097,  2.1551,
         2.5887,  1.8194, -1.0608,  0.4953,  2.1721,  2.8850,  2.2847, -1.2050,
         0.2326,  1.8855,  

In [13]:
#Calculates best_error and expected improvement
best_error = best_error_advanced(model_y, y_exp)[0] #Scaler
best_x = best_error_advanced(model_y, y_exp)[1] #1x3
# print("best x",best_x)
# print(test_p[1])
ei = calc_ei_advanced(best_error,model_mean,model_variance, y_exp) #1x6
print("Best Error is: ",best_error)
print("EI:",ei)
argmax_ei = np.argmax(ei)
print("argmax is", argmax_ei)
print("The test parameter set that gives the highest expected imrovement is \nTheta1, Theta2, x = \n")
print(test_p_mesh_plot[argmax_ei])

Best Error is:  0.0012780247116320543
EI: [4.48892332e-08 7.04418059e-07 4.63149134e-06 8.19390164e-06
 2.26979562e-06 1.95939985e-08 4.06471019e-07 3.26326502e-06
 8.99835141e-06 4.62484893e-06 1.58469168e-08 4.63480365e-07
 3.13041324e-06 8.23511842e-06 7.46253366e-06 7.75442056e-09
 3.91096812e-07 2.46306610e-06 5.86675250e-06 8.98910008e-06
 1.07388827e-09 9.62718520e-08 8.44529432e-07 2.46888807e-06
 6.25643458e-06 8.32440159e-06 8.73296336e-06 8.70405712e-06
 6.93904148e-06 3.98511659e-06 8.74821672e-06 9.21674905e-06
 9.53068472e-06 8.44218445e-06 5.68929011e-06 7.64353734e-06
 7.28238564e-06 8.41813096e-06 8.94472297e-06 7.54456395e-06
 6.49254244e-06 5.61097563e-06 7.55454863e-06 9.47870868e-06
 9.25729867e-06 7.43256320e-06 6.96771920e-06 8.25190534e-06
 9.11966742e-06 8.91422331e-06 8.85046678e-06 9.62334121e-06
 9.37349703e-06 9.57171946e-06 9.47902393e-06 8.37156961e-06
 1.03620526e-05 1.02726476e-05 9.90878645e-06 9.87802986e-06
 5.80264304e-06 8.52024788e-06 1.05910738e-

In [ ]:
y_title = "Model y Value"
print(y_plotter_adv(test_p, model_y, y_title,yval = True))

In [ ]:
print(stdev_plotter_adv(test_p, model_stdev))

In [ ]:
print(ei_plotter_adv(test_p, ei))

In [ ]:
print(error_plotter_adv(test_p, model_y, y_exp))